# Data Science Capstone Project

### Import Required libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

### Acquiring data from wikipedia with BeautifulSoup 

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(source,'html.parser')

### Inputing data into data frame

In [3]:
postal_code=[]
brough=[]
neigh=[]
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postal_code.append(cells[0].text.rstrip('\n'))
        brough.append(cells[1].text.rstrip('\n'))
        neigh.append(cells[2].text.rstrip('\n'))

In [4]:
col= ['Postal Code','Borough','Neighborhood']
df= pd.DataFrame(columns=col)
df['Postal Code']=postal_code
df['Borough']=brough
df['Neighborhood']=neigh
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
pip install geocoder


Note: you may need to restart the kernel to use updated packages.


### Find the Coordinates for each Neighborhood with geocoder

In [20]:
import geocoder
lat=[]
lng=[]
for rows in df:
    i=0
    lat_lng_coords = None
    while(lat_lng_coords is None):
        add = df.at[i,'Postal Code'] + ', Toronto, Ontario'
        g = geocoder.google(add)
        lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])
    i=i+1
print('coordinates found')

Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


KeyboardInterrupt: 

In [ ]:
print(lat)

### Geocoder does not work, instead using provided coordinate data

In [22]:
coor = pd.read_csv('https://cocl.us/Geospatial_data/toronto_coordinates.csv')
coor

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [49]:
i_coor = coor.set_index('Postal Code')
i_df = df.set_index('Postal Code')
df_coor=pd.concat([i_df,i_coor],axis=1,join='inner')
df_coor.tail(10)

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
M1X,Scarborough,Upper Rouge,43.836125,-79.205636
M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [50]:
df_coor.index.name = 'PostalCode'
df_coor.reset_index(inplace=True)
df_coor.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
94,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999
